#데이터 로드 및 전처리
질화규소(layer_1)/이산화규소(layer_2)/질화규소(layer_3)/이산화규소(layer_4)의 두께 예측하기
0~255 : 빛의 파장에 해당하는 반사율 

In [1]:
import math
import time
from itertools import chain
import argparse
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

In [2]:
# 모델 학습을 위해 CUDA 환경 설정 : GPU 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/데이콘 스터디/07.반도체 박막 두께 분석 경진대회/train.csv")
train

,layer_1,layer_2,layer_3,layer_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225
0,10,10,10,10,0.254551,0.258823,0.254659,0.252085,0.247678,0.253614,0.246511,0.259407,0.260862,0.242524,0.253870,0.245156,0.245548,0.255501,0.228948,0.228632,0.225802,0.249418,0.246910,0.248747,0.251088,0.244886,0.233906,0.242632,0.221792,0.236521,0.220555,0.243761,0.230202,0.226122,0.220671,0.235075,0.224560,0.226998,0.209499,0.226594,...,0.160126,0.164192,0.171406,0.162364,0.168591,0.166626,0.169696,0.196383,0.194145,0.190646,0.186797,0.191748,0.190886,0.208518,0.211220,0.229286,0.226265,0.226954,0.229023,0.237112,0.262421,0.262566,0.272062,0.292049,0.305353,0.292889,0.317479,0.316911,0.321371,0.355636,0.354750,0.369223,0.388184,0.408496,0.414564,0.429403,0.419225,0.443250,0.433414,0.465502
1,10,10,10,20,0.205062,0.225544,0.217758,0.202169,0.199633,0.207380,0.191318,0.195369,0.200536,0.197588,0.198726,0.191803,0.199625,0.206465,0.182836,0.193341,0.180459,0.196416,0.185398,0.188913,0.176011,0.173457,0.182249,0.176816,0.192716,0.185924,0.183816,0.186900,0.161040,0.173525,0.172948,0.182634,0.161153,0.158073,0.152335,0.148293,...,0.327076,0.331195,0.337722,0.340189,0.342748,0.351925,0.372989,0.389663,0.396216,0.384977,0.383701,0.403894,0.406382,0.429342,0.409908,0.439307,0.432569,0.439114,0.459541,0.454988,0.475268,0.478575,0.483466,0.496912,0.499542,0.522705,0.524767,0.533952,0.558729,0.572470,0.557203,0.573656,0.587998,0.612754,0.627825,0.633393,0.637706,0.625981,0.653231,0.637853
2,10,10,10,30,0.189196,0.165869,0.177655,0.156822,0.175094,0.177755,0.157582,0.158885,0.156911,0.166162,0.148831,0.144950,0.151362,0.145110,0.159201,0.139296,0.145313,0.156555,0.145363,0.150352,0.150542,0.142990,0.139572,0.145669,0.144514,0.126263,0.112832,0.117535,0.111473,0.117545,0.107058,0.119299,0.110768,0.124217,0.124496,0.110023,...,0.525450,0.532645,0.539097,0.527091,0.531925,0.530325,0.551716,0.556125,0.567470,0.549711,0.566789,0.574313,0.569873,0.574390,0.601672,0.584206,0.602981,0.598708,0.615189,0.637023,0.626458,0.645439,0.629203,0.640967,0.652762,0.660495,0.676498,0.674599,0.693535,0.699182,0.699864,0.708688,0.721982,0.713464,0.743030,0.741709,0.747743,0.746037,0.737356,0.750391
3,10,10,10,40,0.131003,0.120076,0.138975,0.117931,0.130566,0.131262,0.126962,0.134453,0.106717,0.127309,0.099958,0.112908,0.106853,0.108288,0.101393,0.094074,0.086854,0.099669,0.107276,0.091117,0.084525,0.078293,0.076476,0.078559,0.072960,0.075898,0.067963,0.066055,0.087904,0.065373,0.075469,0.080097,0.068895,0.058148,0.057882,0.063258,...,0.616934,0.638654,0.630914,0.647135,0.629796,0.644114,0.641825,0.645360,0.653030,0.656335,0.657734,0.678019,0.676713,0.681540,0.673333,0.686743,0.697500,0.702650,0.693750,0.711013,0.714411,0.701010,0.726972,0.719394,0.726329,0.730821,0.740062,0.747356,0.749405,0.766173,0.764786,0.763788,0.770017,0.787571,0.778866,0.776969,0.774712,0.801526,0.805305,0.784057
4,10,10,10,50,0.091033,0.086893,0.108125,0.080405,0.105917,0.077083,0.097895,0.086765,0.078676,0.075729,0.086023,0.070649,0.078957,0.072772,0.069867,0.080523,0.068273,0.074660,0.069852,0.047442,0.067855,0.049580,0.068737,0.041386,0.048697,0.040278,0.050234,0.059371,0.042581,0.037117,0.041503,0.055113,0.034001,0.035846,0.030385,0.048935,...,0.670888,0.692846,0.673025,0.704448,0.696657,0.697524,0.692513,0.714224,0.696880,0.716361,0.723607,0.720018,0.715039,0.705079,0.720913,0.723440,0.719858,0.729948,0.731360,0.728861,0.740501,0.738362,0.749341,0.738456,0.768791,0.770292,0.766630,0.779970,0.787695,0.766521,0.786677,0.802271,0.806557,0.799614,0.789333,0.804087,0.787763,0.794948,0.819105,0.801781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [4]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/데이콘 스터디/07.반도체 박막 두께 분석 경진대회/test.csv")

In [5]:
#layers = [['layer_1', 'layer_2', 'layer_3', 'layer_4'], [str(i) for i in np.arange(0, 226).tolist()]]
layers = [['layer_1', 'layer_2', 'layer_3', 'layer_4'], [str(i) for i in range(0, 226)]]
layers = list(chain(*layers))

In [6]:
print(train.shape)
train = train.sample(frac=1) #shuffle
rows, cols = train.shape

(810000, 230)


In [8]:
#모델 훈련시 검증을 위해 train 중 10000개를 validation 용으로 분리.
train1 = train.iloc[:rows - 10000,:]
train1 = train1.values #array형
train1 = pd.DataFrame(data=train1,columns=layers) #id를 초기화하기 위해서 진행하는 것 같다.
train1.to_csv('train1.csv', index_label='id')
print("train file saved....")

train file saved....


In [9]:
#validation 파일
val = train.iloc[rows - 10000:,:]
val = val.values
val = pd.DataFrame(data=val,columns=layers)
val.to_csv('val.csv', index_label='id')

In [13]:
# 새로 만든 train/ val 모델 학습 데이터 경로를 설정.
train_path = 'train1.csv'
val_path = 'val.csv'

lr = 1e-03
adam_epsilon = 1e-06
epochs = 100
batch_size = 2048
warmup_step = 2000
loss_fn = nn.L1Loss()


class PandasDataset(Dataset):
  def __init__(self, path):
    super(PandasDataset, self).__init__() #여기서 만든 클래스는 Dataset 클래스의 속성을 가지고 초기화 된다.
    train = pd.read_csv(path).iloc[:, 1:] #id행(index) 제외
    self.train_X, self.train_Y = train.iloc[:,4:], train.iloc[:,0:4]
    self.tmp_x, self.tmp_y = self.train_X.values, self.train_Y.values #array담기

  def __len__(self): #train_X의 크기반환
        return len(self.train_X) 
  
  def __getitem__(self, idx): #idx에 해당하는 값 반환
     return {
            'X':torch.from_numpy(self.tmp_x)[idx],
            'Y':torch.from_numpy(self.tmp_y)[idx]
        }

train_dataset = PandasDataset(train_path)
train_loader = DataLoader(train_dataset, batch_size=batch_size,  num_workers=2)

val_dataset = PandasDataset(val_path)
val_loader = DataLoader(val_dataset, batch_size=batch_size,  num_workers=2) 

- 4개의 layer_1, layer_2, layer_3, layer_4의 두께를 구하는 방법이라 마지막 Dense unit을 1로 해서 각각의 두께를 구하는 모델을 구성해보았으나 결과는 좋지 않음.
- 특별한 탐색적 자료 분석 없음.

# 변수 선택 및 모델 구축
- Learning rate : 다양한 learning rate 스케줄링. 
- Warmup 스케줄링이 train, val loss 줄이는데 가장 효과적.
- Activation function : ReLU, ELU, CeLU, GeLU 사용. 음의 영역에서는 0의 값을 가지는 relu 대신 ELU, CeLU, GeLU등을 사용. relu보다는 효과적.
- Optimizer : adam, adamW.
- Batch size : batch size는 10000으로 시작하였으나 10000보다 작은 값에서 효과적.
- Hidden node, layer depth : node 수는 Nh = Ns/(a * (Ni + No))에 따라 초기 모델링에서는 약 2500개 정도로 하였으나 실험적으로 많은
node가 train, val loss 줄이는데 효과적.
(Nh : number of hidden , Ni : Number of input, No : Number of output, a : 2 - 10)
- Loss funciotion : L1 Loss
- BatchNorm : 모든 layer에 batchNorm 적용
- MLP Model : 단순한 MLP 모델로는 한계를 느껴 MLP의 복잡성을 증가시키는 방향으로 설계
        (1) 노드 수를 증가시켰다 줄이는 방식(UP-block, DOWN-block)
        (2) Down-block에서 skip connection과 layer norm 적용

GELU, LayerNorm, BatchNorm, skipConnectionModel을 사용한다.

In [ ]:
class GELU(nn.Module):
  def forward(self, x):
    return 0.5 * x * (1+torch.tanh(math.sqrt(2/math.pi) * (x+0.044715 * torch.pow(x, 3))))

class LayerNorm(nn.Module):
  def __init__(self, hidden_size, eps=1e-5):
    super(LayerNorm, self).__init__()
    self.weight = nn.Parameter(torch.ones(hidden_size))
    self.bias = nn.Parameter(torch.zeros(hidden_size))
    self.variance_epsilon = eps

    self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias

class skipConnectionModel(nn.Module):
    def __init__(self):
        super(skipConnectionModel, self).__init__()
        #upblock에서는 BatchNorm으로 사용 / downblock에서는 LayerNorm으로 사용한 것 같다.
        self.ln = LayerNorm(10000)
        self.ln1 = LayerNorm(7000)
        self.ln2 = LayerNorm(4000)
        self.ln3 = LayerNorm(2000)

        self.upblock1 = nn.Sequential(nn.Linear(226, 2000),GELU(),nn.BatchNorm1d(2000))
        self.upblock2 = nn.Sequential(nn.Linear(2000,4000),GELU(),nn.BatchNorm1d(4000))
        self.upblock3 = nn.Sequential(nn.Linear(4000,7000), GELU(),nn.BatchNorm1d(7000))
        self.upblock4 = nn.Sequential(nn.Linear(7000,10000),GELU(),nn.BatchNorm1d(10000))

        self.downblock1 = nn.Sequential(nn.Linear(10000, 7000),GELU(),nn.BatchNorm1d(7000))
        self.downblock2 = nn.Sequential(nn.Linear(7000, 4000),GELU(),nn.BatchNorm1d(4000))
        self.downblock3 = nn.Sequential(nn.Linear(4000, 2000),GELU(),nn.BatchNorm1d(2000))
        self.downblock4 = nn.Sequential(nn.Linear(2000, 300),GELU(),nn.BatchNorm1d(300))
        
        self.fclayer = nn.Sequential(nn.Linear(300,4)) #fully connected layer
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        upblock1_out = self.upblock1(x)
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)
        
        downblock1_out = self.downblock1(self.ln(upblock4_out))
        skipblock1 = downblock1_out + upblock3_out
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out
        downblock4_out = self.downblock4(self.ln3(skipblock3))
        
        output = self.fclayer(downblock4_out)
        
        return output
